<a href="https://colab.research.google.com/github/g-zin/twitter-analysis-corona-virus/blob/master/twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!ls '/content/drive/My Drive/Colab Notebooks/twitter.properties'

'/content/drive/My Drive/Colab Notebooks/twitter.properties'


In [4]:
!pip install ConfigParser

In [5]:
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitter.properties')
print(config.sections());

['twitter']


In [6]:
import os 
import tweepy as tw 

In [7]:
accesstoken=config.get('twitter','accesstoken')
accesstokensecret=config.get('twitter','accesstokensecret')
apikey=config.get('twitter','apikey')
apisecretkey=config.get('twitter','apisecretkey')

In [8]:
auth = tw.OAuthHandler(apikey,apisecretkey)
auth.set_access_token(accesstoken,accesstokensecret)
api =tw.API(auth, wait_on_rate_limit=True)

In [9]:
search_words = "#coronavirus"
date_since = "2020-04-01"

In [10]:
tweets = tw.Cursor(api.search, 
                   q= search_words,
                   lang= "en",
                   since= date_since).items(1000)

In [11]:
tweets

In [12]:
tweet_details = [[tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]


In [13]:
import pandas as pd 


In [14]:
tweets_df = pd.DataFrame(data=tweet_details, columns= ['geo', 'text', 'user', 'location'])

In [15]:
pd.set_option('max_colwidth', 800)
tweets_df.head(20) 


,geo,text,user,location
0,None,RT @JoeTalkShow: FYI -- We have NO idea how many Americans have died from #COVID19. Many municipalities are opening admitting if a person…,elitepro,USA
1,None,Ahh...the great mask buying shaming phase of the pandemic. #FlashbackFriday #coronavirus https://t.co/V2QchbSjnW,DrGeniece,"North Carolina, USA"
2,None,NeverForget #NeverForgive\n#StandWithHongKong \n#HongKong \n#China_is_terrorists \n#antichinazi\n#YouCantStopUs… https://t.co/iT3uBB9IU9,11111_iris,
3,None,"Sorry #trump campaign, but the 4.8 mil jobs ""created in June"" as you claim were not ""created""\n\nThey were people goi… https://t.co/panlDEMrF6",TweetyThings1,
4,None,"RT @davepperlmutter: Hey beautiful friends, let’s get #celebsforsmallbiz trending today/tonight! Let’s help the #SmallBiz during these time…",5_littlepiggies,"Nottingham, England"
5,None,"RT @QuickTake: The #coronavirus can spread in the air indoors, the WHO says, a reversal from its previous advice.\n\nMore on #Covid19 @busine…",KatelynRST,Canada
6,None,"RT @cgtnamerica: #Coronavirus was found on shrimp packaging from Ecuador, as China suspends imports from 23 overseas meat companies. \n\nhttp…",elmerfla1,Brasil
7,None,"RT @geonews_english: The latest count of coronavirus cases in Pakistan as of 11:40pm, July 10.\n\n#GeoNews #Coronavirus #Pakistan #StandToget…",sarskhan2,
8,None,RT @delbiz: Join the discussion on how to reopen Delaware's economy safely in this virtual conference presented by @DEStateChamber \nHere's…,DEDeptState,Delaware
9,None,RT @AnaCabrera: NEW: MI Governor Whitmer signs executive order requiring businesses to refuse entry to people not wearing face coverings #c…,Joe_Glidden,United States


In [16]:
tweets_df.user.value_counts()

YngWrldd           19
MEK_Iran           11
Margare59705941     7
TruthSeekers111     7
ChaitralyD          5
                   ..
Manzilgautam        1
genprogress         1
sarah_davies67      1
Mone_Knows          1
SoCalHospitals      1
Name: user, Length: 881, dtype: int64

In [17]:
tweets_df.location.value_counts()

                         295
Fort Lauderdale, FL       20
USA                       15
United States             14
California, USA           13
                        ... 
Gilbert, AZ                1
England, UK                1
Franklin, TN               1
qqqZ                       1
Approx 2m’s away....       1
Name: location, Length: 468, dtype: int64

In [18]:
import re
def clean_tweets(text): 
  text = re.sub("RT @[\w]*:","",text)
  text = re.sub("@[\w]*","",text)
  text = re.sub("https?://[A-Za-z0-9./]*","",text)
  text = re.sub("\n","",text)
  return text 

In [19]:
tweets_df['text']= tweets_df['text'].apply(lambda x : clean_tweets(x))

In [20]:
tweets_df.head(20)

,geo,text,user,location
0,None,FYI -- We have NO idea how many Americans have died from #COVID19. Many municipalities are opening admitting if a person…,elitepro,USA
1,None,Ahh...the great mask buying shaming phase of the pandemic. #FlashbackFriday #coronavirus,DrGeniece,"North Carolina, USA"
2,None,NeverForget #NeverForgive#StandWithHongKong #HongKong #China_is_terrorists #antichinazi#YouCantStopUs…,11111_iris,
3,None,"Sorry #trump campaign, but the 4.8 mil jobs ""created in June"" as you claim were not ""created""They were people goi…",TweetyThings1,
4,None,"Hey beautiful friends, let’s get #celebsforsmallbiz trending today/tonight! Let’s help the #SmallBiz during these time…",5_littlepiggies,"Nottingham, England"
5,None,"The #coronavirus can spread in the air indoors, the WHO says, a reversal from its previous advice.More on #Covid19 …",KatelynRST,Canada
6,None,"#Coronavirus was found on shrimp packaging from Ecuador, as China suspends imports from 23 overseas meat companies. http…",elmerfla1,Brasil
7,None,"The latest count of coronavirus cases in Pakistan as of 11:40pm, July 10.#GeoNews #Coronavirus #Pakistan #StandToget…",sarskhan2,
8,None,Join the discussion on how to reopen Delaware's economy safely in this virtual conference presented by Here's…,DEDeptState,Delaware
9,None,NEW: MI Governor Whitmer signs executive order requiring businesses to refuse entry to people not wearing face coverings #c…,Joe_Glidden,United States


In [21]:
tweets_df.to_csv('tweets.csv')

In [22]:
!ls

drive  sample_data  tweets.csv


In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [24]:
tweets_df['text'].apply(lambda x : [print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

	Text : FYI, Entity : ORG
	Text : Americans, Entity : NORP
	Text : COVID19, Entity : MONEY
	Text : FlashbackFriday, Entity : PERSON
	Text : trump, Entity : MONEY
	Text : 4.8 mil, Entity : QUANTITY
	Text : June, Entity : DATE
	Text : today/tonight, Entity : TIME
	Text : SmallBiz, Entity : PRODUCT
	Text : WHO, Entity : ORG
	Text : Coronavirus, Entity : PERSON
	Text : Ecuador, Entity : GPE
	Text : China, Entity : GPE
	Text : 23, Entity : CARDINAL
	Text : Pakistan, Entity : GPE
	Text : 11:40pm, Entity : CARDINAL
	Text : July 10.#GeoNews, Entity : DATE
	Text : StandToget, Entity : PERSON
	Text : Delaware, Entity : GPE
	Text : Whitmer, Entity : PERSON
	Text : At least 650, Entity : CARDINAL
	Text : 40, Entity : CARDINAL
	Text : Telangana, Entity : GPE
	Text : Hyderabad, Entity : GPE
	Text : One, Entity : CARDINAL
	Text : &amp, Entity : ORG
	Text : Coronavirus, Entity : PERSON
	Text : U.S., Entity : GPE
	Text : first, Entity : ORDINAL
	Text : 500, Entity : CARDINAL
	Text : like &amp, Entity :

0            [None, None, None]
1                        [None]
2                            []
3            [None, None, None]
4                [, None, None]
                 ...           
995    [None, None, None, None]
996                      [None]
997                [None, None]
998                          []
999            [None, None, , ]
Name: text, Length: 1000, dtype: object

In [25]:
tweets_df['entity']=tweets_df['text'].apply(lambda x : [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [26]:
tweets_df.head(20)

,geo,text,user,location,entity
0,None,FYI -- We have NO idea how many Americans have died from #COVID19. Many municipalities are opening admitting if a person…,elitepro,USA,"[(FYI, ORG), (Americans, NORP), (COVID19, MONEY)]"
1,None,Ahh...the great mask buying shaming phase of the pandemic. #FlashbackFriday #coronavirus,DrGeniece,"North Carolina, USA","[(FlashbackFriday, PERSON)]"
2,None,NeverForget #NeverForgive#StandWithHongKong #HongKong #China_is_terrorists #antichinazi#YouCantStopUs…,11111_iris,,[]
3,None,"Sorry #trump campaign, but the 4.8 mil jobs ""created in June"" as you claim were not ""created""They were people goi…",TweetyThings1,,"[(trump, MONEY), (4.8 mil, QUANTITY), (June, DATE)]"
4,None,"Hey beautiful friends, let’s get #celebsforsmallbiz trending today/tonight! Let’s help the #SmallBiz during these time…",5_littlepiggies,"Nottingham, England","[, (today/tonight, TIME), (SmallBiz, PRODUCT)]"
5,None,"The #coronavirus can spread in the air indoors, the WHO says, a reversal from its previous advice.More on #Covid19 …",KatelynRST,Canada,"[, (WHO, ORG)]"
6,None,"#Coronavirus was found on shrimp packaging from Ecuador, as China suspends imports from 23 overseas meat companies. http…",elmerfla1,Brasil,"[(Coronavirus, PERSON), (Ecuador, GPE), (China, GPE), (23, CARDINAL)]"
7,None,"The latest count of coronavirus cases in Pakistan as of 11:40pm, July 10.#GeoNews #Coronavirus #Pakistan #StandToget…",sarskhan2,,"[(Pakistan, GPE), (11:40pm, CARDINAL), (July 10.#GeoNews, DATE), (StandToget, PERSON)]"
8,None,Join the discussion on how to reopen Delaware's economy safely in this virtual conference presented by Here's…,DEDeptState,Delaware,"[(Delaware, GPE)]"
9,None,NEW: MI Governor Whitmer signs executive order requiring businesses to refuse entry to people not wearing face coverings #c…,Joe_Glidden,United States,"[(Whitmer, PERSON)]"


In [27]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

In [28]:
sid = SentimentIntensityAnalyzer()

In [29]:
tweets_df['sentiment'] = tweets_df['text'].apply(lambda x : sid.polarity_scores(x))

In [30]:
tweets_df.head(20)

,geo,text,user,location,entity,sentiment
0,None,FYI -- We have NO idea how many Americans have died from #COVID19. Many municipalities are opening admitting if a person…,elitepro,USA,"[(FYI, ORG), (Americans, NORP), (COVID19, MONEY)]","{'neg': 0.251, 'neu': 0.652, 'pos': 0.097, 'compound': -0.6124}"
1,None,Ahh...the great mask buying shaming phase of the pandemic. #FlashbackFriday #coronavirus,DrGeniece,"North Carolina, USA","[(FlashbackFriday, PERSON)]","{'neg': 0.0, 'neu': 0.709, 'pos': 0.291, 'compound': 0.6249}"
2,None,NeverForget #NeverForgive#StandWithHongKong #HongKong #China_is_terrorists #antichinazi#YouCantStopUs…,11111_iris,,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
3,None,"Sorry #trump campaign, but the 4.8 mil jobs ""created in June"" as you claim were not ""created""They were people goi…",TweetyThings1,,"[(trump, MONEY), (4.8 mil, QUANTITY), (June, DATE)]","{'neg': 0.053, 'neu': 0.831, 'pos': 0.115, 'compound': 0.3291}"
4,None,"Hey beautiful friends, let’s get #celebsforsmallbiz trending today/tonight! Let’s help the #SmallBiz during these time…",5_littlepiggies,"Nottingham, England","[, (today/tonight, TIME), (SmallBiz, PRODUCT)]","{'neg': 0.0, 'neu': 0.546, 'pos': 0.454, 'compound': 0.8748}"
5,None,"The #coronavirus can spread in the air indoors, the WHO says, a reversal from its previous advice.More on #Covid19 …",KatelynRST,Canada,"[, (WHO, ORG)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
6,None,"#Coronavirus was found on shrimp packaging from Ecuador, as China suspends imports from 23 overseas meat companies. http…",elmerfla1,Brasil,"[(Coronavirus, PERSON), (Ecuador, GPE), (China, GPE), (23, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
7,None,"The latest count of coronavirus cases in Pakistan as of 11:40pm, July 10.#GeoNews #Coronavirus #Pakistan #StandToget…",sarskhan2,,"[(Pakistan, GPE), (11:40pm, CARDINAL), (July 10.#GeoNews, DATE), (StandToget, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
8,None,Join the discussion on how to reopen Delaware's economy safely in this virtual conference presented by Here's…,DEDeptState,Delaware,"[(Delaware, GPE)]","{'neg': 0.0, 'neu': 0.735, 'pos': 0.265, 'compound': 0.6597}"
9,None,NEW: MI Governor Whitmer signs executive order requiring businesses to refuse entry to people not wearing face coverings #c…,Joe_Glidden,United States,"[(Whitmer, PERSON)]","{'neg': 0.109, 'neu': 0.891, 'pos': 0.0, 'compound': -0.296}"


In [31]:
!pip install googlemaps

In [32]:
import googlemaps

gmaps = googlemaps.Client(key=config.get('twitter' , 'googleapikey'))
geocode_result = gmaps.geocode(tweets_df['location'][1])
print(geocode_result)
print(geocode_result[0]['formatted_address'])
print(geocode_result[0]['formatted_address'].split(',')[-1].strip())

[{'address_components': [{'long_name': 'North Carolina', 'short_name': 'NC', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'North Carolina, USA', 'geometry': {'bounds': {'northeast': {'lat': 36.5881568, 'lng': -75.400119}, 'southwest': {'lat': 33.7528778, 'lng': -84.32186899999999}}, 'location': {'lat': 35.7595731, 'lng': -79.01929969999999}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 36.5881568, 'lng': -75.400119}, 'southwest': {'lat': 33.7528778, 'lng': -84.32186899999999}}}, 'place_id': 'ChIJgRo4_MQfVIgRGa4i6fUwP60', 'types': ['administrative_area_level_1', 'political']}]
North Carolina, USA
USA


In [33]:
def get_country(input): 
  try:
    output=gmaps.geocode(input)[0]['formatted_address'].split(",")[-1].strip()
  except:
    output="Error"
  return output


In [34]:
tweets_df['country'] = tweets_df['location'].apply(lambda x: "" if (not x.strip()) else get_country(x))

In [35]:
tweets_df['country'].value_counts()


USA                             354
                                295
UK                               78
India                            51
Error                            40
United States                    35
Canada                           18
Australia                        16
Iran                             13
South Africa                      9
Nigeria                           8
Germany                           8
Pakistan                          7
United Kingdom                    6
Denmark                           5
Hong Kong                         5
Brazil                            5
Ireland                           4
Mexico                            4
Spain                             3
Chile                             3
Sweden                            3
Netherlands                       2
United Arab Emirates              2
Israel                            2
Saudi Arabia                      2
Japan                             2
Maldives                    

In [37]:
tweets_df.head(20)

,geo,text,user,location,entity,sentiment,country
0,None,FYI -- We have NO idea how many Americans have died from #COVID19. Many municipalities are opening admitting if a person…,elitepro,USA,"[(FYI, ORG), (Americans, NORP), (COVID19, MONEY)]","{'neg': 0.251, 'neu': 0.652, 'pos': 0.097, 'compound': -0.6124}",United States
1,None,Ahh...the great mask buying shaming phase of the pandemic. #FlashbackFriday #coronavirus,DrGeniece,"North Carolina, USA","[(FlashbackFriday, PERSON)]","{'neg': 0.0, 'neu': 0.709, 'pos': 0.291, 'compound': 0.6249}",USA
2,None,NeverForget #NeverForgive#StandWithHongKong #HongKong #China_is_terrorists #antichinazi#YouCantStopUs…,11111_iris,,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",
3,None,"Sorry #trump campaign, but the 4.8 mil jobs ""created in June"" as you claim were not ""created""They were people goi…",TweetyThings1,,"[(trump, MONEY), (4.8 mil, QUANTITY), (June, DATE)]","{'neg': 0.053, 'neu': 0.831, 'pos': 0.115, 'compound': 0.3291}",
4,None,"Hey beautiful friends, let’s get #celebsforsmallbiz trending today/tonight! Let’s help the #SmallBiz during these time…",5_littlepiggies,"Nottingham, England","[, (today/tonight, TIME), (SmallBiz, PRODUCT)]","{'neg': 0.0, 'neu': 0.546, 'pos': 0.454, 'compound': 0.8748}",UK
5,None,"The #coronavirus can spread in the air indoors, the WHO says, a reversal from its previous advice.More on #Covid19 …",KatelynRST,Canada,"[, (WHO, ORG)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Canada
6,None,"#Coronavirus was found on shrimp packaging from Ecuador, as China suspends imports from 23 overseas meat companies. http…",elmerfla1,Brasil,"[(Coronavirus, PERSON), (Ecuador, GPE), (China, GPE), (23, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Brazil
7,None,"The latest count of coronavirus cases in Pakistan as of 11:40pm, July 10.#GeoNews #Coronavirus #Pakistan #StandToget…",sarskhan2,,"[(Pakistan, GPE), (11:40pm, CARDINAL), (July 10.#GeoNews, DATE), (StandToget, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",
8,None,Join the discussion on how to reopen Delaware's economy safely in this virtual conference presented by Here's…,DEDeptState,Delaware,"[(Delaware, GPE)]","{'neg': 0.0, 'neu': 0.735, 'pos': 0.265, 'compound': 0.6597}",USA
9,None,NEW: MI Governor Whitmer signs executive order requiring businesses to refuse entry to people not wearing face coverings #c…,Joe_Glidden,United States,"[(Whitmer, PERSON)]","{'neg': 0.109, 'neu': 0.891, 'pos': 0.0, 'compound': -0.296}",United States


In [38]:
tweets_df.dtypes

geo          object
text         object
user         object
location     object
entity       object
sentiment    object
country      object
dtype: object

In [41]:
tweets_df.to_csv('tweets.csv')

In [42]:
!ls


drive  sample_data  tweets.csv
